In [1]:
!pip -q install ultralytics boxmot fvcore ffmpeg-python opencv-python-headless dash Pillow plotly parse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 139.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
import os, csv, json, cv2, math, time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any, DefaultDict, Sequence
from collections import defaultdict

from tqdm.auto import tqdm
from pathlib import Path
from IPython.display import Video

import ffmpeg
from ultralytics import YOLO
from boxmot import StrongSort

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import random

def seed_all(seed=1023):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_all(2310)

# **PATHS**

In [5]:
# rally = "asy_wzy_rally_1"
rally = "shi_vit_rally_1"
# rally = "syq_anton_rally_1"

INPUT_VIDEO_PATH = f"/content/{rally}.mp4"

PLAYER_TRACKS_CSV_PATH = f"/content/{rally}_tracks.csv"

SHUTTLE_CSV_PATH = f"/content/{rally}_ball.csv"

YOLO_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/YOLO/my_yolo11s_finals.pt"

REID_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/REID/osnet_x1_0_badminton.pt"

SLOWFAST_PATH = "/content/drive/MyDrive/FIT3163,3164/SlowFast/07_models/slowfast_finals_1b.pt"

# **Step 1: STRONGSORT**

In [6]:
@dataclass
class YoloCfg:
    conf: float = 0.35
    iou: float = 0.5                      # intersection over union threshold: the I:U ratio that is considered as correct
    imgsz: int = 1280
    max_det: int = 300
    classes: Optional[Sequence[int]] = None   # e.g. [0] for COCO 'person'
    agnostic_nms: bool = False
    verbose: bool = False

@dataclass
class StrongSortCfg:
    # Only applied if attributes exist on your StrongSort build (safe setattr).
    max_age: Optional[int] = 40           # keep ID alive across occlusion
    n_init: Optional[int] = 3             # frames before confirming a track
    max_iou_dist: Optional[float] = 0.7   # motion/overlap gating
    max_dist: Optional[float] = 0.2       # appearance/cosine distance gating
    nn_budget: Optional[int] = 100        # feature queue size
    half: bool = True                     # use FP16 on GPU if available
    det_thresh: float = 0.3

In [7]:
def load_models(yolo_weights: str, reid_weights: str, ssort_cfg: StrongSortCfg):
    device = 0 if torch.cuda.is_available() else 'cpu'
    tracker = StrongSort(
        reid_weights=Path(reid_weights),
        device=device,
        half=(ssort_cfg.half and torch.cuda.is_available()),
        # BaseTracker parameters
        det_thresh=ssort_cfg.det_thresh,
        max_age=ssort_cfg.max_age,
        n_init=ssort_cfg.n_init,
        max_iou_dist=ssort_cfg.max_iou_dist,
        max_cos_dist=ssort_cfg.max_dist,
        nn_budget=ssort_cfg.nn_budget
    )
    print(f"StrongSort args: {vars(tracker)}")
    print(f"StrongSort Tracker args: {vars(tracker.tracker)}")

    yolo = YOLO(yolo_weights)
    return yolo, tracker

def yolo_detect(yolo: YOLO, frame: np.ndarray, cfg: YoloCfg):
    r = yolo.predict(
        frame,
        conf=cfg.conf,
        iou=cfg.iou,
        imgsz=cfg.imgsz,
        max_det=cfg.max_det,
        classes=cfg.classes,
        agnostic_nms=cfg.agnostic_nms,
        verbose=cfg.verbose
    )[0]
    if r.boxes is None or r.boxes.xyxy.numel() == 0:
        return None
    boxes = r.boxes.xyxy.detach().cpu().numpy()  # [N,4]
    confs = r.boxes.conf.detach().cpu().numpy()  # [N]
    clss  = r.boxes.cls.detach().cpu().numpy()   # [N]
    dets  = np.concatenate([boxes, confs[:, None], clss[:, None]], axis=1)
    return dets  # [x1,y1,x2,y2,conf,cls]

def draw_tracks(frame: np.ndarray, tracks: np.ndarray, id_color_cache: dict):
    # tracks: Nx [x1,y1,x2,y2,track_id,conf,cls,ind]
    for tb in tracks:
        x1, y1, x2, y2, tid, conf, cls, _ = tb
        x1,y1,x2,y2,tid = int(x1),int(y1),int(x2),int(y2),int(tid)
        color = id_color_cache.setdefault(tid, (37*tid%256, 17*tid%256, 93*tid%256))
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return frame

def export_tracks_csv(csv_path: str, rows: List[Tuple[int,int,int,int,int,int,float,int]]):
    # rows: (frame, id, x1,y1,x2,y2, conf, cls)
    if not csv_path: return
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["frame","id","x1","y1","x2","y2","conf","cls"])
        w.writerows(rows)

In [8]:
def track_video(
    src_path: str,
    dst_path: str,
    yolo: YOLO,
    tracker: StrongSort,
    yolo_cfg: YoloCfg,
    export_csv: Optional[str] = None,
    show_pbar: bool = True,
):
    cap = cv2.VideoCapture(src_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {src_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    w, h = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter(dst_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or None
    print(f"🎥 Video Meta: FPS={fps:.2f}, Total Frames={total}")
    pbar = tqdm(total=total, unit="frame", disable=not show_pbar)

    csv_rows = []
    id_color_cache = {}
    frame_idx = 0

    t0 = time.time()
    while True:
        ok, frame = cap.read()
        if not ok: break

        dets = yolo_detect(yolo, frame, yolo_cfg)
        if dets is None:
            out.write(frame)
            frame_idx += 1
            pbar.update(1)
            continue

        tracks = tracker.update(dets, frame)  # -> Nx [x1,y1,x2,y2,track_id,conf,cls,ind]
        if tracks is not None and len(tracks):
            # collect CSV rows
            for x1,y1,x2,y2,tid,conf,cls,_ in tracks:
                csv_rows.append((frame_idx, int(tid), int(x1), int(y1), int(x2), int(y2), float(conf), int(cls)))
            # draw
            frame = draw_tracks(frame, tracks, id_color_cache)

        cv2.putText(
            frame, text=f"Frame {frame_idx}", org=(50, 200),
            fontScale=1.3, color=(0, 0, 255), thickness=4, fontFace=cv2.FONT_HERSHEY_SIMPLEX
        )

        out.write(frame)
        frame_idx += 1
        pbar.update(1)

    cap.release(); out.release()
    pbar.close()
    t1 = time.time()
    print(f"Saved video to: {dst_path} | frames={frame_idx} | time={t1-t0:.1f}s | ~{frame_idx/max(t1-t0,1):.1f} FPS")
    if export_csv:
        export_tracks_csv(export_csv, csv_rows)
        print(f"Saved track CSV to: {export_csv}")

# **Step 2: Track Shuttle**

In [9]:
!git clone https://github.com/qaz812345/TrackNetV3.git

# Get model checkpoints
!gdown --fuzzy https://drive.google.com/file/d/1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA/view?usp=sharing
!unzip /content/TrackNetV3_ckpts.zip
!mv ckpts TrackNetV3/ckpts

Cloning into 'TrackNetV3'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 240 (delta 99), reused 87 (delta 87), pack-reused 129 (from 1)
Receiving objects: 100% (240/240), 2.82 MiB | 62.67 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Downloading...
From (original): https://drive.google.com/uc?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA
From (redirected): https://drive.google.com/uc?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA&confirm=t&uuid=59ba954c-0bc6-4883-bb7e-7b916f594273
To: /content/TrackNetV3_ckpts.zip
100% 132M/132M [00:03<00:00, 36.8MB/s]
Archive:  /content/TrackNetV3_ckpts.zip
   creating: ckpts/
  inflating: ckpts/InpaintNet_best.pt  
  inflating: ckpts/TrackNet_best.pt  


# **Step 3: Get Contact Points**

In [10]:
@dataclass
class ContactConfig:
    """Configuration for dataset preparation"""
    window_size: int = 21  # frames to look at around each point (odd number). 21 means 10 before and 10 after
    temporal_stride: int = 5  # stride for negative sampling (reduce dataset size)
    positive_window: int = 5  # frames around true contact to mark as positive

    # Feature engineering
    compute_velocity: bool = True
    compute_acceleration: bool = True
    compute_direction: bool = True
    smooth_window: int = 5

class ContactDetector:
    """Inference class for detecting contact points from shuttle CSV"""

    def __init__(
        self,
        model: nn.Module,
        cfg: ContactConfig,
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    ):
        self.model = model.to(device)
        self.model.eval()
        self.device = device
        self.cfg = cfg

    def predict_csv(
        self,
        csv_path: str,
        confidence_threshold: float = 0.5,
        nms_window: int = 10
    ) -> List[int]:
        """
        Predict contact frames from shuttle CSV.

        Args:
            csv_path: Path to shuttle CSV file
            confidence_threshold: Minimum confidence for detection
            nms_window: Non-maximum suppression window (frames)

        Returns:
            List of contact frame numbers
        """
        # Load CSV
        frames, vis, xs, ys = self._load_csv(csv_path)

        # Extract features
        features = self._extract_features(xs, ys, vis)

        # Sliding window inference
        probabilities = self._sliding_window_inference(features)

        # Map probabilities back to frame numbers
        half_window = self.cfg.window_size // 2
        valid_frames = frames[half_window:-half_window]

        # Apply threshold and NMS
        contact_frames = self._detect_peaks(
            valid_frames, probabilities, confidence_threshold, nms_window
        )

        return contact_frames

    def _load_csv(self, path: str):
        """Load shuttle CSV (same as dataset)"""
        frames, vis, xs, ys = [], [], [], []
        with open(path, 'r') as f:
            reader = csv.DictReader(f)
            for r in reader:
                frames.append(int(r["Frame"]))
                vis.append(int(r["Visibility"]))
                xs.append(float(r["X"]))
                ys.append(float(r["Y"]))

        frames = np.array(frames)
        vis = np.array(vis)
        xs = np.array(xs, dtype=np.float32)
        ys = np.array(ys, dtype=np.float32)

        idx = np.argsort(frames)
        return frames[idx], vis[idx], xs[idx], ys[idx]

    def _extract_features(self, xs, ys, vis):
        """Extract features (same as dataset)"""
        # [Same implementation as in ShuttleContactDataset]
        xs_filled = xs.copy()
        ys_filled = ys.copy()
        valid = (vis == 1) & ~((xs == 0) & (ys == 0))

        for i in range(1, len(xs)):
            if not valid[i]:
                xs_filled[i] = xs_filled[i-1]
                ys_filled[i] = ys_filled[i-1]

        def smooth(arr, window):
            if window < 2:
                return arr
            if window % 2 == 0:
                window += 1
            pad = window // 2
            padded = np.pad(arr, pad, mode='edge')
            kernel = np.ones(window) / window
            return np.convolve(padded, kernel, mode='valid').astype(np.float32)

        xs_s = smooth(xs_filled, self.cfg.smooth_window)
        ys_s = smooth(ys_filled, self.cfg.smooth_window)

        features = [xs_s, ys_s]

        vx = np.diff(xs_s, prepend=xs_s[0])
        vy = np.diff(ys_s, prepend=ys_s[0])
        speed = np.sqrt(vx**2 + vy**2)
        features.extend([vx, vy, speed])

        ax = np.diff(vx, prepend=vx[0])
        ay = np.diff(vy, prepend=vy[0])
        features.extend([ax, ay])

        direction = np.arctan2(vy, vx)
        direction_change = np.diff(direction, prepend=direction[0])
        features.extend([direction, direction_change])

        return np.stack(features, axis=1)

    def _sliding_window_inference(self, features):
        """Run model on sliding windows"""
        N = len(features)
        half_window = self.cfg.window_size // 2
        probabilities = []

        with torch.no_grad():
            for i in range(half_window, N - half_window):
                start_idx = i - half_window
                end_idx = i + half_window + 1
                window = features[start_idx:end_idx]

                # Prepare input
                window_tensor = torch.tensor(window, dtype=torch.float32).T.unsqueeze(0)
                window_tensor = window_tensor.to(self.device)

                # Predict
                output = self.model(window_tensor)
                prob = F.softmax(output, dim=1)[0, 1].item()  # probability of contact
                probabilities.append(prob)

        return np.array(probabilities)

    def _detect_peaks(self, frames, probabilities, threshold, nms_window):
        """Detect peaks in probability curve with NMS"""
        # Threshold
        candidates = []
        for i, prob in enumerate(probabilities):
            if prob >= threshold:
                candidates.append((frames[i], prob))

        if not candidates:
            return []

        # Sort by probability (descending)
        candidates.sort(key=lambda x: x[1], reverse=True)

        # NMS: keep peaks that are far enough apart
        selected = []
        for frame, prob in candidates:
            # Check if too close to already selected peaks
            too_close = any(abs(frame - sel) < nms_window for sel in selected)
            if not too_close:
                selected.append(frame)

        return sorted(selected)

In [11]:
class ContactDetectionCNN(nn.Module):
    """
    1D CNN for temporal contact detection.

    Architecture:
    - Multiple Conv1D layers to capture temporal patterns
    - Residual connections for gradient flow
    - Global pooling to handle variable-length sequences
    - Binary classification head
    """

    def __init__(self, input_features: int, hidden_dim: int = 64):
        super().__init__()

        # Feature extraction layers
        self.conv1 = nn.Conv1d(input_features, hidden_dim, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(hidden_dim)

        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(hidden_dim)

        self.conv3 = nn.Conv1d(hidden_dim, hidden_dim * 2, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(hidden_dim * 2)

        self.conv4 = nn.Conv1d(hidden_dim * 2, hidden_dim * 2, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm1d(hidden_dim * 2)

        # Global pooling + classification
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 2)  # binary classification
        )

    def forward(self, x):
        # x: (batch, features, time)

        # Conv blocks with residuals
        x1 = F.relu(self.bn1(self.conv1(x)))
        x2 = F.relu(self.bn2(self.conv2(x1)))
        x2 = x2 + x1  # residual

        x3 = F.relu(self.bn3(self.conv3(x2)))
        x4 = F.relu(self.bn4(self.conv4(x3)))
        x4 = x4 + x3  # residual

        # Global pool and classify
        x_pooled = self.pool(x4).squeeze(-1)  # (batch, hidden*2)
        logits = self.fc(x_pooled)  # (batch, 2)

        return logits

In [12]:
# best_contact_model_v3.pth
!gdown --fuzzy https://drive.google.com/file/d/1OLCLAL6FtZCcHK6tY9wJ1G2o0VtfQSxb/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1OLCLAL6FtZCcHK6tY9wJ1G2o0VtfQSxb
To: /content/best_contact_model_v3.pth
100% 441k/441k [00:00<00:00, 131MB/s]


# **Utilities**

In [13]:
def _device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def _linspace_idx(a: int, b: int, n: int) -> List[int]:
    if n <= 1 or b <= a:
        return [a] * max(n, 1)
    return np.round(np.linspace(a, b, n)).astype(int).tolist()

def _expand_to_square(x1, y1, x2, y2, W, H, factor=1.25):
    cx, cy = 0.5*(x1+x2), 0.5*(y1+y2)
    w, h = (x2-x1), (y2-y1)
    s = max(w, h) * factor
    nx1 = int(max(0, math.floor(cx - 0.5*s)))
    ny1 = int(max(0, math.floor(cy - 0.5*s)))
    nx2 = int(min(W-1, math.ceil (cx + 0.5*s)))
    ny2 = int(min(H-1, math.ceil (cy + 0.5*s)))
    if nx2 <= nx1: nx2 = min(W-1, nx1+1)
    if ny2 <= ny1: ny2 = min(H-1, ny1+1)
    return nx1, ny1, nx2, ny2

def _color_for_id(tid: int):
    # stable-ish distinct color per track id
    return (37*tid % 256, 17*tid % 256, 93*tid % 256)

# **Load Tracks CSV**

In [14]:
def load_tracks_df(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df["frame"] = df["frame"].astype(int)
    df["id"] = df["id"].astype(int)
    for c in ("x1","y1","x2","y2"):
        df[c] = df[c].astype(float)
    return df

def per_player_lookup(df: pd.DataFrame) -> Dict[int, Dict[int, Tuple[float,float,float,float]]]:
    look = {}
    for pid in df["id"].unique():
        sub = df[df["id"] == pid][["frame","x1","y1","x2","y2"]]
        look[int(pid)] = {int(r.frame):(float(r.x1),float(r.y1),float(r.x2),float(r.y2)) for r in sub.itertuples(index=False)}
    return look

def nearest_bbox(ff: int, lookup: Dict[int, Tuple[float,float,float,float]], span: int=3):
    if ff in lookup:
        return lookup[ff]
    for d in range(1, span+1):
        if ff-d in lookup: return lookup[ff-d]
        if ff+d in lookup: return lookup[ff+d]
    return None

def gather_dense_boxes(pid_lookup: Dict[int, Tuple[float,float,float,float]],
                       t0: int, t1: int, span: int=3) -> Optional[List[Tuple[int,int,int,int]]]:
    boxes = []
    for f in range(t0, t1+1):
        bb = nearest_bbox(f, pid_lookup, span=span)
        if bb is None:
            return None
        x1,y1,x2,y2 = bb
        boxes.append((int(x1),int(y1),int(x2),int(y2)))
    return boxes

# **Step 4: Sampling with SlowFast**

In [15]:
@dataclass
class SlowFastInferCfg:
    slow_t: int = 8
    alpha: int = 4
    side: int = 224
    mean: Tuple[float,float,float] = (0.45, 0.45, 0.45)
    std:  Tuple[float,float,float] = (0.225, 0.225, 0.225)
    bbox_margin: float = 1.3
    bbox_ema: float = 0.8

In [16]:
def sample_indices(L: int, slow_t: int, alpha: int):
    need_fast = slow_t * alpha
    idx_fast = _linspace_idx(0, max(0, L-1), need_fast)
    idx_slow = idx_fast[::alpha]
    if len(idx_slow) < slow_t:
        idx_slow += [idx_slow[-1]] * (slow_t - len(idx_slow))
    elif len(idx_slow) > slow_t:
        idx_slow = idx_slow[:slow_t]
    return idx_slow, idx_fast

def extract_clip_slowfast(frames_dict: Dict[int, np.ndarray],
                          segment: Tuple[int,int],
                          dense_boxes: List[Tuple[int,int,int,int]],
                          sfcfg: SlowFastInferCfg):
    """
    frames_dict: {frame_idx -> BGR frame}
    segment: inclusive [t0, t1]
    dense_boxes: per-frame (x1,y1,x2,y2) aligned to t0..t1 (same length as segment)
    """
    t0, t1 = segment
    L = t1 - t0 + 1
    if L <= 0 or len(dense_boxes) < L:
        return torch.empty(0), torch.empty(0)

    # Smooth bbox sequence (EMA)
    smoothed = []
    prev = None
    for b in dense_boxes[:L]:
        arr = np.array(b, dtype=np.float32)
        prev = arr if prev is None else sfcfg.bbox_ema * prev + (1.0 - sfcfg.bbox_ema) * arr
        smoothed.append(tuple(prev.astype(int)))

    # Collect crops
    # infer frame shape from any available frame
    any_im = frames_dict.get(t0, None)
    if any_im is None:
        return torch.empty(0), torch.empty(0)
    H, W = any_im.shape[:2]

    crops = []
    for k in range(L):
        fidx = t0 + k
        fr = frames_dict.get(fidx)
        if fr is None:
            # pad by repeating last valid frame
            fr = crops[-1] if len(crops) else any_im

        x1,y1,x2,y2 = smoothed[k]
        sx1, sy1, sx2, sy2 = _expand_to_square(x1, y1, x2, y2, W, H, factor=sfcfg.bbox_margin)
        crop = fr[sy1:sy2, sx1:sx2]
        if crop.size == 0:
            crop = fr
        crop = cv2.resize(crop, (sfcfg.side, sfcfg.side), interpolation=cv2.INTER_AREA)
        crops.append(crop)

    idx_slow, idx_fast = sample_indices(len(crops), sfcfg.slow_t, sfcfg.alpha)

    def _pack(frames_bgr_idx):
        if not frames_bgr_idx:
            return torch.empty(0)
        arr = np.stack([cv2.cvtColor(crops[i], cv2.COLOR_BGR2RGB) for i in frames_bgr_idx]).astype(np.float32)/255.0
        arr = (arr - np.array(sfcfg.mean)) / np.array(sfcfg.std)
        # (T,H,W,C) -> (C,T,H,W)
        return torch.from_numpy(np.transpose(arr, (3,0,1,2))).float()

    slow = _pack(idx_slow)
    fast = _pack(idx_fast)
    return slow, fast

# **Load SlowFast**

In [17]:
class SlowFastPredictor:
    def __init__(self, model, device=None):
        self.model = model.eval()
        self.device = device or _device()

    @torch.no_grad()
    def predict_batch(self, slow_list: List[torch.Tensor], fast_list: List[torch.Tensor]) -> np.ndarray:
        if not slow_list:
            return np.empty((0,0))
        slow = torch.stack(slow_list).to(self.device)
        fast = torch.stack(fast_list).to(self.device)
        logits = self.model([slow, fast])
        return torch.softmax(logits, dim=1).cpu().numpy()

In [18]:
def load_slowfast_classifier(cfg, ckpt_path: str, device: Optional[torch.device] = None):
    device = device or _device()
    print(f"🧠 Loading SlowFast model from: {ckpt_path}")
    torch.hub._validate_not_a_forked_repo = lambda a,b,c: True
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r101', pretrained=True)
    in_dim = model.blocks[-1].proj.in_features
    model.blocks[-1].proj = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(in_dim, 128),
        nn.Dropout(p=0.3), # Add a dropout layer
        nn.Linear(128, len(cfg.labels))
    )
    ckpt = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(ckpt["model"], strict=True)
    model.eval().to(device)
    print(f"   -> Model loaded on {device}")
    return model

# **Main function**

In [19]:
@dataclass
class InferenceCfg:
    labels: List[str]
    sf: SlowFastInferCfg
    n_before_after: int = 10      # window size: [c-n, c+n]
    search_span: int = 3          # bbox nearest fill +/- frames

In [20]:
@dataclass
class TrainCfg:
    labels = [
        "block", "clear", "cross_net",
        "drive", "drop", "jump_smash",
        "lift", "push", "serve",
        "smash", "straight_net", "tap",
        "negative"
    ]

In [21]:
def run_full_video_to_events(
        video_path: str,
        tracks_csv: str,
        contact_frames: List[int],
        model,
        cfg: InferenceCfg
    ) -> Dict:
    """
    Returns:
      {
        "events": [
            {"track_id": int, "t0": int, "t1": int, "label": str, "p": float},
            ...
        ],
        "contacts": contact_frames
      }
    """
    # 1) Read tracks and build lookups
    df = load_tracks_df(tracks_csv)
    print(f"Loaded player tracks CSV.")
    by_pid = per_player_lookup(df)
    player_ids = sorted(by_pid.keys())

    # 2) Open video and cache any frames we touch
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}.")

    frame_cache: Dict[int, np.ndarray] = {}
    def get_frame(fidx: int) -> Optional[np.ndarray]:
        if fidx in frame_cache:
            return frame_cache[fidx]
        if fidx < 0 or fidx >= total_frames:
            return None
        cap.set(cv2.CAP_PROP_POS_FRAMES, fidx)
        ok, fr = cap.read()
        if not ok:
            return None
        frame_cache[fidx] = fr
        return fr

    # 3) For efficiency, prefetch all frames needed by all contacts
    needed = set()
    for c in contact_frames:
        t0 = max(0, c - cfg.n_before_after)
        t1 = min(total_frames - 1, c + cfg.n_before_after)
        needed.update(range(t0, t1+1))
    for fidx in tqdm(sorted(needed), desc="Pre-fetching frames: "):
        _ = get_frame(fidx)

    # 4) Build batches & run model
    predictor = SlowFastPredictor(model, device=_device())
    slow_batch, fast_batch, metas = [], [], []

    for c in tqdm(contact_frames, desc="Processing contact frames: "):
        t0 = c - cfg.n_before_after
        t1 = c + cfg.n_before_after
        if t0 < 0 or t1 >= total_frames:
            # skip if outside bounds
            continue

        # Create frames_dict for this window
        frames_dict = {f: frame_cache.get(f) for f in range(t0, t1+1)}
        # For each player present in tracks
        for pid in player_ids:
            # Gather dense bboxes for this pid across window
            dense = gather_dense_boxes(by_pid[pid], t0, t1, span=cfg.search_span)
            if dense is None:
                continue

            slow_t, fast_t = extract_clip_slowfast(frames_dict, (t0, t1), dense, cfg.sf)
            if slow_t.numel() == 0 or fast_t.numel() == 0:
                continue

            slow_batch.append(slow_t)
            fast_batch.append(fast_t)
            metas.append((pid, t0, t1))

    events = []
    if slow_batch:
        print(f"Predicting batch with SlowFast.")
        probs = predictor.predict_batch(slow_batch, fast_batch)
        for (pid, t0, t1), p in (zip(metas, probs)):
            k = int(np.argmax(p))
            events.append({
                "track_id": int(pid),
                "t0": int(t0),
                "t1": int(t1),
                "label": cfg.labels[k],
                "p": float(p[k]),
            })

    cap.release()
    print(f"Completed.")
    events.sort(key=lambda e: (e["t0"], e["track_id"]))
    return {"events": events, "contacts": [int(c) for c in contact_frames]}

# **Step 5: Overlay on video**

In [22]:
def load_events_and_contacts(events_json_path: str):
    """Loads an events JSON file."""
    with open(events_json_path, "r") as f:
        data = json.load(f)
    return data.get("events", data), data.get("contacts", data)

def build_event_map_by_frame(events: List[Dict]) -> Dict[int, Dict[int, Dict]]:
    """Converts a list of events into a per-frame lookup: {frame: {track_id: event}}."""
    by_frame: DefaultDict[int, Dict[int, Dict]] = defaultdict(dict)
    for e in events:
        for fr in range(int(e["t0"]), int(e["t1"]) + 1):
            by_frame[fr][int(e["track_id"])] = e
    return by_frame

def load_tracks_csv_overlay(csv_path: str):
    """
    Returns:
      boxes_by_frame: {frame_idx: [(tid, (x1,y1,x2,y2)) ...]}
    """
    boxes_by_frame = defaultdict(list)
    with open(csv_path, "r") as f:
        r = csv.DictReader(f)
        for row in r:
            fi = int(row["frame"])
            tid = int(row["id"])
            x1,y1,x2,y2 = int(row["x1"]), int(row["y1"]), int(row["x2"]), int(row["y2"])
            boxes_by_frame[fi].append((tid, (x1,y1,x2,y2)))
    return boxes_by_frame

In [23]:
def render_full_video_overlay(
    video_frames: List[np.ndarray], # MODIFIED: Accepts frames list
    tracks_csv: str,
    shuttle_csv: str,
    events_json: str,
    out_path: str,
    fps: float,                     # ADDED: Accepts fps
    show_ids: bool = True,
    label_bg_alpha: float = 0.4
):
    boxes_by_frame = defaultdict(list)
    with open(tracks_csv, "r") as f:
        for r in csv.DictReader(f):
            boxes_by_frame[int(r["frame"])].append((int(r["id"]), (int(r["x1"]), int(r["y1"]), int(r["x2"]), int(r["y2"]))))

    shuttle_coordinates = defaultdict(list)
    with open(shuttle_csv, "r") as f2:
        for r in csv.DictReader(f2):
            shuttle_coordinates[int(r["Frame"])].extend([int(r["X"]), int(r["Y"])])

    events, contacts = load_events_and_contacts(events_json)
    event_by_frame = build_event_map_by_frame(events)

    # Get video dimensions from the first frame
    if not video_frames:
        print("Warning: video_frames list is empty. Cannot generate overlay.")
        return
    H, W, _ = video_frames[0].shape

    scale = max(H, W) / 1080.0

    BOX_TH       = max(3, int(round(6 * scale)))           # colored box thickness
    OUTLINE_TH   = BOX_TH + max(2, int(round(2 * scale)))  # black outline thickness
    FONT_SCALE   = max(0.8, 0.9 * scale)                   # text size
    TXT_TH       = max(2, int(round(3 * scale)))           # text thickness (white fill)
    STROKE_TH    = TXT_TH + max(1, int(round(2 * scale)))  # black text stroke
    PAD          = max(6, int(round(8 * scale)))           # label padding
    BG_ALPHA     = max(label_bg_alpha, 0.65)               # stronger translucent label bg

    # --- FFMPEG LOGIC ---
    # MODIFIED: Uses the passed 'fps' parameter
    process = (
        ffmpeg
        .input('pipe:', format='rawvideo', pix_fmt='bgr24', s=f'{W}x{H}', r=fps)
        .output(
            out_path,
            vcodec='libx264',
            pix_fmt='yuv420p',
            acodec='aac',
            strict='experimental'
        )
        .overwrite_output()
        .run_async(pipe_stdin=True)
    )

    try:
        # MODIFIED: Loop over the in-memory frames list
        for i, frame in tqdm(enumerate(video_frames), desc="Writing video: "):
            overlay_frame = frame.copy()

            # Draw shuttlecock
            shuttle_color = _color_for_id(5)
            cv2.circle(overlay_frame, center=(shuttle_coordinates[i][0], shuttle_coordinates[i][1]),
                       radius=10, thickness=5, color=shuttle_color)

            # Draw frame number
            cv2.putText(overlay_frame, text=f"Frame {i}", org=(50, 200),
                        fontScale=FONT_SCALE, color=(255, 255, 200), thickness=4, fontFace=cv2.FONT_HERSHEY_SIMPLEX)

            # Draw note for contact frames
            if i in contacts:
                cv2.putText(overlay_frame, text="Contact", org=(50, 250),
                            fontScale=FONT_SCALE*1.2, color=(0, 0, 255), thickness=4, fontFace=cv2.FONT_HERSHEY_SIMPLEX)

            # Draw bounding boxes
            for tid, (x1,y1,x2,y2) in boxes_by_frame.get(i, []):
                color = _color_for_id(tid)

                # ---- BOLDER BOX: black outline + colored inner, AA lines ----
                cv2.rectangle(overlay_frame, (x1, y1), (x2, y2), (0, 0, 0), OUTLINE_TH, lineType=cv2.LINE_AA)
                cv2.rectangle(overlay_frame, (x1, y1), (x2, y2), color, BOX_TH, lineType=cv2.LINE_AA)

                # ---- LABEL / ID ----
                ev = event_by_frame.get(i, {}).get(tid)

                # Only draw text if shot is predicted AND is not negative
                if ev and ev['label'] != "negative":
                    text = f"{ev['label']} {ev['p']:.2f}"
                elif show_ids:
                    text = f"ID {tid}"
                else:
                    text = None

                if text:
                    # Size text
                    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, TXT_TH)

                    # Place label above box if possible, else below
                    tx = x1 + BOX_TH
                    ty = y1 - PAD
                    if ty - th - PAD < 0:
                        ty = y1 + th + PAD

                    # Semi-transparent color pill behind text
                    bg = overlay_frame.copy()
                    cv2.rectangle(
                        bg,
                        (tx - PAD,          ty - th - int(1.2 * PAD)),
                        (tx + tw + PAD,     ty + int(0.6 * PAD)),
                        color,
                        -1
                    )
                    overlay_frame = cv2.addWeighted(bg, BG_ALPHA, overlay_frame, 1 - BG_ALPHA, 0)

                    # High-contrast text: black stroke + white fill
                    cv2.putText(overlay_frame, text, (tx, ty),
                                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (0, 0, 0),
                                STROKE_TH, lineType=cv2.LINE_AA)

                    cv2.putText(overlay_frame, text, (tx, ty),
                                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (255, 255, 255),
                                TXT_TH, lineType=cv2.LINE_AA)

            process.stdin.write(overlay_frame.tobytes())

    finally:
        if process.stdin:
            process.stdin.close()
        process.wait()
        print(f"FFmpeg process finished. Video saved to {out_path}")

# **RUN FULLLL FLOW**

### these are counted in the initialising stage

In [ ]:
yolo_model, ssort_tracker = load_models(YOLO_MODEL_PATH, REID_MODEL_PATH, ssort_cfg)

In [ ]:
contact_model = ContactDetectionCNN(input_features=9, hidden_dim=64)
contact_model.load_state_dict(torch.load('best_contact_model_v3.pth'))
print(f"ContactDetectionCNN created with {sum(p.numel() for p in contact_model.parameters()):,} parameters.")

In [ ]:
device = _device()
slowfast_model = load_slowfast_classifier(cfg=trainconfig, ckpt_path=SLOWFAST_PATH, device=device)

In [ ]:
cap = cv2.VideoCapture(str(INPUT_VIDEO_PATH))
video_fps = cap.get(cv2.CAP_PROP_FPS)

all_frames = []
while True:
    ok, frame = cap.read()
    if not ok:
        break
    all_frames.append(frame)

### each video being processed should only go through the code below

# **Step 1: StrongSort (YOLO + ReID)**

In [24]:
"""
max_age: The maximum number of consecutive frames a track can exist without being matched to a new detection before it is deleted.
n_init: The minimum number of consecutive detections required before a track is considered "confirmed" (and ready to be output).
max_iou_dist: The maximum Intersection-over-Union (IoU) distance allowed for a match. Higher values allow a detection to be matched to a track even if it overlaps LESS.
max_dist: Higher values allow more distant detections (spatially or in appearance feature space) to be matched to a track.
nn_budget: The maximum number of appearance features (from the deep neural network) to store for each track.
half: probably to use half precision
"""
ssort_cfg = StrongSortCfg(
    max_age=30,          # keep IDs alive over occlusions/net crossings # default: 50
    n_init=12,             # ori: 3
    max_iou_dist=1,     # ori: 0.7
    max_dist=1,        # ori: 0.25
    nn_budget=240,        # ori: 120
    det_thresh=0.4,
    half=True
)

yolo_cfg = YoloCfg(
    conf=0.50,           # a bit lower to catch partial players, ori: 0.30
    iou=0.8,             # ori: 0.55
    imgsz=1280,
    max_det=2,          # ori: 200
    classes=None,        # e.g., [0] if your model uses COCO 'person'
    agnostic_nms=False,
    verbose=False
)

In [26]:
OUTPUT_VIDEO_PATH = f"/content/{rally}_tracks.mp4"
CSV_PATH = f"/content/{rally}_tracks.csv"

track_video(INPUT_VIDEO_PATH, OUTPUT_VIDEO_PATH, yolo_model, ssort_tracker, yolo_cfg, export_csv=CSV_PATH, show_pbar=True)

2025-11-03 14:59:04.703 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:56 | __init__ - BaseTracker initialization parameters:
2025-11-03 14:59:04.704 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:57 | __init__ - det_thresh: 0.4
2025-11-03 14:59:04.704 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:58 | __init__ - max_age: 30
2025-11-03 14:59:04.704 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:59 | __init__ - max_obs: 50
2025-11-03 14:59:04.704 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:60 | __init__ - min_hits: 3
2025-11-03 14:59:04.704 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:61 | __init__ - iou_threshold: 

StrongSort args: {'det_thresh': 0.4, 'max_age': 30, 'max_obs': 50, 'min_hits': 3, 'iou_threshold': 0.3, 'per_class': False, 'nr_classes': 80, 'asso_func_name': 'iou', 'is_obb': False, 'frame_count': 0, 'active_tracks': [], 'per_class_active_tracks': None, '_first_frame_processed': False, '_first_dets_processed': False, 'last_emb_size': None, 'min_conf': 0.1, 'model': <boxmot.appearance.backends.pytorch_backend.PyTorchBackend object at 0x79949241e000>, 'tracker': <boxmot.trackers.strongsort.sort.tracker.Tracker object at 0x7994937ef770>, 'cmc': <boxmot.motion.cmc.ecc.ECC object at 0x79949c187e00>}
StrongSort Tracker args: {'metric': <boxmot.trackers.strongsort.sort.linear_assignment.NearestNeighborDistanceMetric object at 0x7994934dc1a0>, 'max_iou_dist': 1, 'max_age': 30, 'n_init': 12, '_lambda': 0, 'ema_alpha': 0.9, 'mc_lambda': 0.98, 'tracks': [], '_next_id': 1, 'cmc': <boxmot.motion.cmc.ecc.ECC object at 0x7994934223c0>}
🎥 Video Meta: FPS=30.00, Total Frames=781


  0%|          | 0/781 [00:00<?, ?frame/s]

Saved video to: /content/shi_vit_rally_1_tracks.mp4 | frames=781 | time=59.4s | ~13.1 FPS
Saved track CSV to: /content/shi_vit_rally_1_tracks.csv


# **Step 2: Track Shuttle**

In [28]:
!python TrackNetV3/predict.py --video_file $INPUT_VIDEO_PATH --tracknet_file TrackNetV3/ckpts/TrackNet_best.pt --inpaintnet_file TrackNetV3/ckpts/InpaintNet_best.pt --save_dir ./

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100% 49/49 [00:51<00:00,  1.05s/it]
100% 48/48 [00:00<00:00, 56.68it/s]
Done.


# **Step 3: Get Contact Frames**

In [29]:
contactconfig = ContactConfig(
    window_size=21,
    temporal_stride=5,
    positive_window=4  # 4 frames before and 4 frames after will be marked as positive
)

contact_detector = ContactDetector(contact_model, contactconfig)

ContactDetectionCNN created with 106,626 parameters.


In [30]:
contact_frames = contact_detector.predict_csv(
    SHUTTLE_CSV_PATH,
    confidence_threshold=0.6,
    nms_window=10
)

CONTACT_FRAMES = [int(frame) for frame in contact_frames]
print(f"Detected {len(CONTACT_FRAMES)} contact frames: {CONTACT_FRAMES}")

Detected 26 contact frames: [90, 114, 138, 158, 195, 209, 229, 261, 277, 306, 329, 354, 383, 409, 446, 461, 485, 512, 526, 553, 582, 604, 666, 687, 727, 758]


# **Step 4: Extract Clips and Run SlowFast Inference**

In [31]:
trainconfig = TrainCfg()

slowfastinferconfig = SlowFastInferCfg(
    side=224,
    bbox_margin=1.3,
    bbox_ema=0.8
)

inferenceconfig = InferenceCfg(
    labels=trainconfig.labels,
    sf=slowfastinferconfig,
    n_before_after=15,
    search_span=3
)

🧠 Loading SlowFast model from: /content/drive/MyDrive/FIT3163,3164/SlowFast/07_models/slowfast_finals_1b.pt
Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOWFAST_8x8_R101.pyth" to /root/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R101.pyth


100%|██████████| 480M/480M [00:27<00:00, 18.3MB/s]


   -> Model loaded on cuda


In [32]:
result = run_full_video_to_events(
    video_path=INPUT_VIDEO_PATH,
    tracks_csv=PLAYER_TRACKS_CSV_PATH,
    contact_frames=CONTACT_FRAMES,
    model=slowfast_model,
    cfg=inferenceconfig
)

with open(f"{Path(INPUT_VIDEO_PATH).stem}.json", "w") as f:
    json.dump(result, f, indent=2)
print(f"Predicted {len(result['events'])} action events")

Loaded player tracks CSV.
Total frames in video: 781.


Pre-fetching frames:   0%|          | 0/646 [00:00<?, ?it/s]

Processing contact frames:   0%|          | 0/26 [00:00<?, ?it/s]

Predicting batch with SlowFast.
Completed.
Predicted 52 action events


# **Step 5: Overlay to Video**

In [33]:
EVENTS_JSON = f'{Path(INPUT_VIDEO_PATH).stem}.json'

render_full_video_overlay(
    video_frames=all_frames,
    tracks_csv=PLAYER_TRACKS_CSV_PATH,
    shuttle_csv=SHUTTLE_CSV_PATH,
    events_json=EVENTS_JSON,
    out_path=f"/content/{Path(INPUT_VIDEO_PATH).stem}_overlay.mp4",
    fps=video_fps
)

Writing video: : 0it [00:00, ?it/s]

FFmpeg process finished. Video saved to /content/shi_vit_rally_1_overlay.mp4
